In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch torchvision torchaudio
!pip install --upgrade torch torchvision torchaudio


import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
import torchvision.transforms as transforms

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score


# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
#import timm
#model = timm.create_model('tf_efficientnet_b0', checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth')
#model.eval()

#import urllib
#from PIL import Image
#from timm.data import resolve_data_config
#from timm.data.transforms_factory import create_transform

In [ ]:
def calculate_f1(preds, labels, is_multiclass=True, average_type='weighted'):
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()
    if is_multiclass:
        # Для многоклассовой классификации
        return f1_score(labels, preds, average=average_type)
    else:
        # Для бинарной классификации
        return f1_score(labels, preds, average='binary')

In [ ]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TRAIN_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'

In [ ]:
CONFIG = {
    "seed": 40,
    "epochs": 3,
    "img_size": 512,
    "model_name": "tf_efficientnet_b0_ns",
    "checkpoint_path" : "/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth",
    "num_classes": 5,
    "train_batch_size": 32,
    "valid_batch_size": 64,
    "learning_rate": 1e-4,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [ ]:
import torch
import torch.nn as nn
import timm

class CustomModel(nn.Module):
    def __init__(self, model_name=CONFIG['model_name'], num_classes=CONFIG['num_classes'], pretrained=True):
        super(CustomModel, self).__init__()
        # Загружаем предобученную модель из timm
        self.model = timm.create_model(model_name, pretrained=pretrained)
        
        # Получаем количество признаков на входе последнего слоя
        if 'resnet' in model_name or 'efficientnet' in model_name:
            n_features = self.model.get_classifier().in_features
        elif 'vit' in model_name or 'deit' in model_name:
            n_features = self.model.head.in_features
        else:
            # Добавьте поддержку других архитектур по необходимости
            raise NotImplementedError("Model architecture not supported yet.")
        
        # Заменяем последний слой модели на новый, с выходом по количеству классов
        # для задачи классификации
        if 'resnet' in model_name or 'efficientnet' in model_name:
            self.model.fc = nn.Linear(n_features, num_classes)
        elif 'vit' in model_name or 'deit' in model_name:
            self.model.head = nn.Linear(n_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
image_paths = ROOT_DIR

In [ ]:
def get_train_file_path(image_id):
    return f"{TRAIN_DIR}/{image_id}_thumbnail.png"

In [ ]:
train_images = sorted(glob.glob(f"{TRAIN_DIR}/*.png"))


In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df['file_path'] = df['image_id'].apply(get_train_file_path)
df = df[ df["file_path"].isin(train_images) ].reset_index(drop=True)
df

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        target = self.targets[index]
        return image, target
    
    
    
train_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

valid_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [ ]:
#train_dataset = CustomDataset(TRAIN_DIR, train_targets, transform=train_transforms)
#valid_dataset = CustomDataset(ROOT_DIR, valid_targets, transform=valid_transforms)

#train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], shuffle=True)
#valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], shuffle=False)

In [ ]:
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

with open("label_encoder.pkl", "wb") as fp:
    joblib.dump(encoder, fp)

In [ ]:
CONFIG['T_max'] = df.shape[0] * (CONFIG["n_fold"]-1) * CONFIG['epochs'] // CONFIG['train_batch_size'] // CONFIG["n_fold"]
CONFIG['T_max']

In [ ]:
#CONFIG = {
#    "seed": 42,
#    "epochs": 20,
#    "img_size": 512,
#    "model_name": "tf_efficientnet_b0_ns",
#    "checkpoint_path" : "/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth",
#    "num_classes": 8,
#    "train_batch_size": 32,
#    "valid_batch_size": 64,
#    "learning_rate": 1e-4,
#    "scheduler": 'CosineAnnealingLR',
#    "min_lr": 1e-6,
#    "T_max": 500,
#    "weight_decay": 1e-6,
#    "fold" : 0,
#    "n_fold": 5,
#    "n_accumulate": 1,
#    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#}

In [ ]:
def set_seed(seed=40):

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.label)):
      df.loc[val_ , "kfold"] = int(fold)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        target = self.targets[index]
        return image, target

In [ ]:
train_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(),
])

valid_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(),
])


In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
from sklearn.metrics import precision_score, recall_score

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    running_loss = 0.0
    correct_predictions = torch.tensor(0, device=device)  # Используем тензор для подсчета
    dataset_size = 0
    all_preds = []
    all_labels = []

    for step, data in tqdm(enumerate(dataloader), total=len(dataloader)):
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.long)
        batch_size = images.size(0)
        
        optimizer.zero_grad()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Обновление после каждого батча

        running_loss += (loss.item() * batch_size)
        correct_predictions += (predicted == labels).sum()
        dataset_size += batch_size
        
        all_preds.extend(predicted.view(-1).cpu().numpy())
        all_labels.extend(labels.view(-1).cpu().numpy())

    epoch_loss = running_loss / dataset_size
    epoch_acc = correct_predictions.double() / dataset_size
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    epoch_precision = precision_score(all_labels, all_preds, average='macro')
    epoch_recall = recall_score(all_labels, all_preds, average='macro')

    return epoch_loss, epoch_acc.item(), epoch_f1, epoch_precision, epoch_recall





def valid_one_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in tqdm(dataloader):
            inputs = data['image'].to(device)
            labels = data['label'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.data.view(-1).cpu().numpy())

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
    epoch_precision = precision_score(all_labels, all_preds, average='weighted')
    epoch_recall = recall_score(all_labels, all_preds, average='weighted')

    print(f'Validation Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, F1: {epoch_f1:.4f}, Precision: {epoch_precision:.4f}, Recall: {epoch_recall:.4f}')

    return epoch_loss, epoch_acc, epoch_f1, epoch_precision, epoch_recall


In [ ]:
#model = CustomModel().to(CONFIG['device'])
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])


#train_transforms = A.Compose([
#    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#    ToTensorV2(),
#])

#valid_transforms = A.Compose([
#    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#    ToTensorV2(),
#])

In [ ]:
#for epoch in range(CONFIG['epochs']):
#    print(f'Epoch {epoch+1}/{CONFIG["epochs"]}')
#    train_one_epoch(model, train_loader, criterion, optimizer, CONFIG['device'])
#    valid_one_epoch(model, valid_loader, criterion, CONFIG['device'])
#    print('-' * 10)

In [ ]:
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True, checkpoint_path=None):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, checkpoint_path=checkpoint_path)

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        return output

    
model = UBCModel(CONFIG['model_name'], CONFIG['num_classes'], checkpoint_path=CONFIG['checkpoint_path'])
model.to(CONFIG['device']);

In [ ]:
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

In [ ]:
#def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
#    model.train()
#    running_loss = 0.0
#    correct_predictions = 0
#    all_preds = []
#    all_targets = []
    
#    for inputs, labels in dataloader:
#        inputs = inputs.to(device)
#        labels = labels.to(device)
#        
#        optimizer.zero_grad()
#        
#        outputs = model(inputs)
#        loss = criterion(outputs, labels)
#        loss.backward()
#        optimizer.step()
        
#        running_loss += loss.item() * inputs.size(0)
#        _, preds = torch.max(outputs, 1)
#        correct_predictions += torch.sum(preds == labels)
        
        # Collect all predictions and labels for F1 score calculation
#        all_preds.append(preds)
#        all_targets.append(labels)
        
    # Convert all collected predictions and labels to a single tensor
#    all_preds = torch.cat(all_preds).cpu()
#    all_targets = torch.cat(all_targets).cpu()
    
    # Calculate accuracy
#    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    
    # Calculate F1 score using sklearn's f1_score function
#    epoch_f1 = f1_score(all_targets.numpy(), all_preds.numpy(), average='macro')
    
#    epoch_loss = running_loss / len(dataloader.dataset)
    
#    return epoch_loss, epoch_acc, epoch_f1


In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in tqdm(dataloader):
            inputs = data['image'].to(device, dtype=torch.float)
            labels = data['label'].to(device, dtype=torch.long)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    epoch_precision = precision_score(all_labels, all_preds, average='macro')
    epoch_recall = recall_score(all_labels, all_preds, average='macro')

    print(f'Validation Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, F1: {epoch_f1:.4f}, Precision: {epoch_precision:.4f}, Recall: {epoch_recall:.4f}')

    return epoch_loss, epoch_acc, epoch_f1, epoch_precision, epoch_recall

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_acc = -np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_acc, train_epoch_f1, train_epoch_precision, train_epoch_recall = train_one_epoch(model, optimizer, 
                                                                            scheduler, 
                                                                            dataloader=train_loader, 
                                                                            device=CONFIG['device'], epoch=epoch)


        
        val_epoch_loss, val_epoch_acc, val_epoch_f1, val_epoch_precision, val_epoch_recall = valid_one_epoch(model, valid_loader, criterion, 
                                                                  CONFIG['device'])
                                                                 #epoch=epoch)
        
        #train_loss, train_acc, train_f1 = train_one_epoch(...)
        #valid_loss, valid_acc, valid_f1 = valid_one_epoch(...)

        #history['Train F1'].append(train_f1)
        #history['Valid F1'].append(valid_f1)
    
        #history['Train Loss'].append(train_epoch_loss)
        #history['Valid Loss'].append(val_epoch_loss)
        #history['Train Accuracy'].append(train_epoch_acc)
        #history['Valid Accuracy'].append(val_epoch_acc)
        
        history['Train Loss'].append(train_epoch_loss)
        history['Train Accuracy'].append(train_epoch_acc)
        history['Train F1'].append(train_epoch_f1)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Accuracy'].append(val_epoch_acc)
        history['Valid F1'].append(val_epoch_f1)
        
        history['Train Precision'].append(train_epoch_precision)
        history['Train Recall'].append(train_epoch_recall)
        history['Valid Precision'].append(val_epoch_precision)
        history['Valid Recall'].append(val_epoch_recall)


        history['lr'].append( scheduler.get_lr()[0] )
        
        # deep copy the model
        if best_epoch_acc <= val_epoch_acc:
            print(f"{b_}Validation Accuracy Improved ({best_epoch_acc} ---> {val_epoch_acc})")
            best_epoch_acc = val_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "Acc{:.2f}_Loss{:.4f}_epoch{:.0f}.bin".format(best_epoch_acc, val_epoch_loss, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Accuracy: {:.4f}".format(best_epoch_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = UBCDataset(df_train, transforms=data_transforms["train"])
    valid_dataset = UBCDataset(df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)

    return train_loader, valid_loader


In [ ]:
train_loader, valid_loader = prepare_loaders(df, fold=CONFIG["fold"])
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

In [ ]:
model, history = run_training(model, optimizer, scheduler, device=CONFIG['device'], num_epochs=CONFIG['epochs'])

In [ ]:
history = pd.DataFrame.from_dict(history)
history.to_csv("history.csv", index=False)
history.to_csv("submission.csv", index=False)

In [ ]:
plt.plot( range(history.shape[0]), history["Train Loss"].values, label="Train Loss")
plt.plot( range(history.shape[0]), history["Valid Loss"].values, label="Valid Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["Train Accuracy"].values, label="Train Accuracy")
plt.plot( range(history.shape[0]), history["Valid Accuracy"].values, label="Valid Accuracy")
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.grid()
plt.legend()
plt.show()

In [ ]:
best_metrics = {
    'Best Valid F1': history['Valid F1'][best_f1_index],
    'Corresponding Valid Accuracy': history['Valid Accuracy'][best_f1_index],
    'Corresponding Valid Precision': history['Valid Precision'][best_f1_index],
    'Corresponding Valid Recall': history['Valid Recall'][best_f1_index]
}

best_metrics